In [2]:
#Import libraries
import pandas as pd

In [3]:
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler

In [4]:
# Load your CSV
df = pd.read_csv("./2/dataset_sun_wind.csv").dropna()

In [5]:
df = df[df["period"] == "train_a"]

In [6]:
def parse_time(time_str):
    days, time = time_str.split(' days ')
    hours, minutes, seconds = map(int, time.split(':'))
    return pd.Timedelta(days=int(days), hours=hours, minutes=minutes, seconds=seconds)


df["timedelta"] = df["timedelta"].apply(parse_time)

In [7]:
# Create a fake datetime starting point
start_time = pd.Timestamp("2020-01-01")
df["timestamp"] = start_time + pd.to_timedelta(df["timedelta"])

In [8]:
# Create TimeSeries objects
target_series = TimeSeries.from_dataframe(df, 
                                          time_col="timestamp", 
                                          value_cols="dst", 
                                          freq="T")  # target only

past_covariates = TimeSeries.from_dataframe(df,
                                            time_col="timestamp", 
                                            value_cols=["bt", 
                                                        "density",
                                                        "temperature",
                                                        "bx_gse",
                                                        "by_gse",
                                                        "bz_gse",
                                                        "gse_x",
                                                        "gse_y", 
                                                        "gse_z", 
                                                        "speed"],
                                            freq="T")  # inputs

In [9]:
# Normalize
scaler_target = Scaler()
scaler_covariates = Scaler()

In [10]:
target_series = scaler_target.fit_transform(target_series)
past_covariates = scaler_covariates.fit_transform(past_covariates)

In [11]:
# Train/test split
train_target = target_series[:-50]
val_target = target_series[-50:]

train_covs = past_covariates[:-50]
val_covs = past_covariates[-50:]

In [16]:
from darts.models import TCNModel

/Users/andrea/opt/anaconda3/lib/python3.9/site-packages/threadpoolctl.py:1226: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [31]:
model = TCNModel(
    input_chunk_length=60,
    output_chunk_length=10,
    n_epochs=100,
    random_state=42,
    dropout=0.1,
)

In [32]:
model.fit(train_target, past_covariates=train_covs)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | res_blocks      | ModuleList       | 332    | train
-------------------------------------------------------------
332       Trainable params
0         Non-trainable params
332       Total params
0.001     Total estimated model params size (MB)
28        Modules in train mode
0         Modules in eval mode


Training: |                                                            | 0/? [00:00<?, ?it/s]

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
`Trainer.fit` stopped: `max_epochs=100` reached.


TCNModel(output_chunk_shift=0, kernel_size=3, num_filters=3, num_layers=None, dilation_base=2, weight_norm=False, dropout=0.1, input_chunk_length=60, output_chunk_length=10, n_epochs=100, random_state=42)

In [33]:
forecast = model.predict(
    n=10,
    past_covariates=val_covs,
)

# Inverse scale
forecast = scaler_target.inverse_transform(forecast)
val_target = scaler_target.inverse_transform(val_target)

# Plot
val_target.plot(label="True")
forecast.plot(label="Predicted")

ValueError: For the given forecasting case, the provided `past_covariates` at series sequence index `0` do not extend far enough into the past. The `past_covariates` must start at or before time step `2020-04-07 01:22:00`, whereas now the start is at time step `2020-04-07 02:22:00`.


ValueError: For the given forecasting case, the provided `past_covariates` at series sequence index `0` do not extend far enough into the past. The `past_covariates` must start at or before time step `2020-04-07 01:22:00`, whereas now the start is at time step `2020-04-07 02:22:00`.

In [26]:
import numpy as np
i = 0
val = train_target.all_values()
for el in val:
    if np.isnan(el[0][0]):
        i+=1
i

62951